In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import swifter
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score, explained_variance_score, mean_absolute_error
from statsmodels.base.model import GenericLikelihoodModel

/home/opotowsky/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [18]:
lbls = ["a", "b", "c", "d", "e"]
pred_lbls = ["Pred_" + s for s in lbls]
dict(zip(lbls, pred_lbls))

{'a': 'Pred_a', 'b': 'Pred_b', 'c': 'Pred_c', 'd': 'Pred_d', 'e': 'Pred_e'}

In [36]:
pred_lbls.append('test col')
pred_lbls

['Pred_a', 'Pred_b', 'Pred_c', 'Pred_d', 'Pred_e', 'test col', 'test col']

In [3]:
def vary(values, frac=0.1, round_dec=2):
    dev = frac * np.random.uniform(-1, 1, len(values))
    new_vals = np.array(values) * (1 + dev)
    new_vals = np.round_(new_vals, round_dec)
    return new_vals

In [4]:
#make trainDB
ones = np.ones(20)
meas1 = vary(ones)
meas4 = vary(1.5 * ones)
meas2 = vary(2 * ones)
meas5 = vary(2.5 * ones)
meas3 = vary(3 * ones)

trainXY = pd.DataFrame({"a" : meas1, 
                        "b" : meas4,
                        "c" : meas2,
                        "d" : meas5,
                        "e" : meas3
                        })
X = trainXY.copy() # make a copy

trainXY

,a,b,c,d,e
0,1.08,1.40,1.90,2.47,2.88
1,1.10,1.63,2.00,2.28,2.83
2,0.95,1.37,1.93,2.46,3.22
3,1.09,1.50,2.12,2.31,3.23
4,0.99,1.62,2.05,2.41,3.09
5,0.92,1.53,2.14,2.70,2.92
6,0.90,1.40,2.04,2.74,3.03
7,1.03,1.38,2.06,2.31,2.77
8,1.03,1.39,2.09,2.32,3.23
9,0.94,1.42,1.95,2.54,3.08


In [5]:
#make test cases
test1 = [1, 1.5, 2, 2.5, 3] # best case
test2 = [1, 0, 0, 0, 0] # 1 meas only, worst-ish case 
test3 = [1, 1, 1, 1, 1] # bad measurements
test4 = [0, 1.5, 2.5, 0, 0] # 2 meas only
test5 = [1, 0, 2, 0, 3]  # 3 meas only
tests = [test1, test2, test3, test4, test5]

testXY = pd.DataFrame(tests, columns = ["a", "b", "c", "d", "e"])
testXY

,a,b,c,d,e
0,1,1.5,2.0,2.5,3
1,1,0.0,0.0,0.0,0
2,1,1.0,1.0,1.0,1
3,0,1.5,2.5,0.0,0
4,1,0.0,2.0,0.0,3


In [6]:
def ll_calc(y_sim, y_mes, std):
    ll = np.sum(stats.norm.logpdf(y_sim, loc=y_mes, scale=std))
    return ll

def unc_calc(y_sim, y_mes, sim_unc_sq, mes_unc_sq):
    unc = ((y_sim - y_mes) / sim_unc_sq)**2 * (sim_unc_sq + mes_unc_sq)
    unc.replace([np.inf, -np.inf], 0, inplace=True)
    unc.fillna(0, inplace = True)
    return np.sqrt(unc.sum(axis=1))

In [27]:
def calc_ll(XY, test_sample, unc):
    ll_name = 'LogLikelihood_' + str(unc)
    #X = XY.copy()#drop(lbls, axis=1).copy()
    XY[ll_name] = X.apply(lambda row: ll_calc(row, test_sample, unc*row), axis=1)
    max_ll = XY[ll_name].max()
    max_idx = XY[ll_name].idxmax()
    pred_answer = XY.loc[XY.index == max_idx].drop(ll_name, axis=1)
    lbls = ["a", "b", "c", "d", "e"]
    pred_lbls = ["Pred_" + s for s in lbls]
    pred_answer = pred_answer.rename(columns=dict(zip(lbls, pred_lbls)))
    
    some_cols = ["Pred_a", "Pred_c", 'Pred_e']
    test_slice = pred_answer.loc[:, some_cols]
    print(test_slice)
    
    return max_ll, max_idx, pred_answer

In [20]:
y_true = testXY.index.to_list()
y_true

[0, 1, 2, 3, 4]

In [28]:
#%%time

unc = 0.1
y_pred = []
lls = []
for i, t in enumerate(tests):
    colname = 'LogLikelihood_' + str(i+1)
    uncname = 'LLUncertainty_' + str(i+1)
    test_sample = testXY.loc[testXY.index == i]
    # In this case, the idx will be the labels
    max_ll, max_idx, pred_sample = calc_ll(trainXY, test_sample, unc)
    #trainXY[colname] = X.apply(lambda row: ll_calc(row, test_sample, unc*row), axis=1)
    #trainXY[uncname] = X.apply(lambda row: unc_calc(row, test_sample, (unc*row)**2, (unc*test_sample)**2), axis=1)
    y_pred.append(max_idx)
    lls.append(max_ll)
    if i == 0:
        pred_df = pd.DataFrame(columns = pred_sample.columns.to_list())
    pred_df = pred_df.append(pred_sample)

   Pred_a  Pred_c  Pred_e
9    0.94    1.95    3.08
   Pred_a  Pred_c  Pred_e
7    1.03    2.06    2.77
   Pred_a  Pred_c  Pred_e
7    1.03    2.06    2.77
    Pred_a  Pred_c  Pred_e
19    0.98    2.18    3.09
    Pred_a  Pred_c  Pred_e
16    0.97    1.95    3.16


In [17]:
colnames = ['LL', 'True Label', 'Pred Label']
pred_df[colnames] = pd.DataFrame(columns=[lls, y_true, y_pred], index=pred_df.index)
pred_df

ValueError: Columns must be same length as key

In [13]:
all_tests = pd.DataFrame({'True Label' : y_true, 'Pred Label' : y_pred, 'Loglikelihood' : lls})
all_tests

,True Label,Pred Label,Loglikelihood
0,0,9,3.463112
1,1,7,-196.054627
2,2,7,-49.580061
3,3,19,-147.391490
4,4,16,-96.284445


In [14]:
all_tests['Absolute Error'] = all_tests.apply(lambda row: np.abs(row[0]-row[1]), axis=1)
all_tests

,True Label,Pred Label,Loglikelihood,Absolute Error
0,0,9,3.463112,9.0
1,1,7,-196.054627,6.0
2,2,7,-49.580061,5.0
3,3,19,-147.391490,16.0
4,4,16,-96.284445,12.0


In [9]:
trainXY

,a,b,c,d,e,LogLikelihood_1,LLUncertainty_1,LogLikelihood_2,LLUncertainty_2,LogLikelihood_3,LLUncertainty_3,LogLikelihood_4,LLUncertainty_4,LogLikelihood_5,LLUncertainty_5
0,1.06,1.38,2.20,2.30,3.14,2.343329,2.407131,-196.387908,20.015133,-54.252762,11.713839,-147.535533,17.490290,-96.900527,14.230083
1,0.91,1.35,1.88,2.45,3.20,2.495724,2.570125,-196.467143,20.053911,-51.929443,11.550861,-152.033337,18.244644,-96.866168,14.274516
2,1.09,1.62,1.89,2.59,2.81,2.654644,2.050438,-196.612671,20.031365,-54.612253,11.828622,-151.754559,18.156597,-97.010633,14.246120
3,1.07,1.37,2.13,2.44,2.93,2.903415,1.929615,-196.401352,20.020035,-53.229867,11.638417,-148.146309,17.582780,-96.616141,14.199283
4,0.94,1.49,2.05,2.31,2.78,3.116693,1.954016,-196.199918,20.021701,-51.303042,11.555097,-148.407739,17.663347,-96.542793,14.224633
5,0.93,1.60,1.91,2.29,2.87,2.878174,2.166462,-196.292437,20.030516,-51.766872,11.622872,-150.975456,18.072725,-96.506041,14.216765
6,0.94,1.42,2.00,2.51,3.00,3.554205,1.242275,-196.286303,20.021701,-53.478402,11.684354,-149.366290,17.795707,-96.286303,14.172809
7,1.08,1.44,2.10,2.31,3.08,2.925987,1.847681,-196.501834,20.025462,-53.772132,11.709756,-148.128350,17.582150,-96.648945,14.197994
8,1.03,1.35,2.01,2.56,2.71,2.692506,2.363100,-196.088938,20.004120,-50.549105,11.406600,-149.635266,17.829619,-96.662744,14.237914
9,0.93,1.55,1.94,2.46,3.11,3.396180,1.384072,-196.428193,20.030516,-55.089518,11.857602,-150.363176,17.954698,-96.538571,14.200718


In [38]:
max_pwr = trainXY['LogLikelihood'].loc[trainXY['ReactorType'] == 'pwr'].max()
idx_pwr = trainXY['LogLikelihood'].loc[trainXY['ReactorType'] == 'pwr'].idxmax()
unc_pwr = float(trainXY['LLUncertainty'].loc[trainXY.index == idx_pwr])
print(f'Max Log Likelihood for PWRs: {max_pwr} +/- {unc_pwr}')
trainXY.loc[trainXY.index == idx_pwr, ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']]

Max Log Likelihood for PWRs: -172.46966586074987 +/- 228.7371919493878


,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
1013,pwr,99.56,3.1,1854.07,ce14x14


#### AGR